In [1]:
%load_ext autoreload
%autoreload 2
import random
from cap.dehaze import dehaze as cap_dehaze
from dcp.dehaze import dehaze as dcp_dehaze
from config import config
# from measure import compute_psnr, compute_ssim
import os
from glob import glob
from matplotlib import pyplot as plt
import cv2
from sklearn.metrics import mean_squared_error as compute_mse
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim
from tqdm import tqdm
import timeit
import numpy as np

In [2]:
_RESIDE_PATH = 'D:\\data\\reside\\indoor_train'
_RESULT_PATH = '../result'

In [3]:
hazy_fold_path = os.path.join(_RESIDE_PATH,'hazy')
clear_fold_path = os.path.join(_RESIDE_PATH,'clear')

In [4]:
def get_clear(hazy_img_path, clear_fold_path):
    hazy_img_path = hazy_img_path.replace('\\', '/')
    clear_file = hazy_img_path.split('/')[-1].split('_')[0] + '.png'
    clear_img_path = os.path.join(clear_fold_path, clear_file)
    return clear_img_path

In [5]:
def show_compaire(hazy, clear, result_path, save = False):
    """
    all is the files
    """
    hazy_img = cv2.imread(hazy)
    clear_img = cv2.imread(clear)
    
    dehaze_1 = dcp_dehaze(hazy, config['dcp'])
    dehaze_2 = cap_dehaze(hazy, config['cap'])
    
    psnr_1 = compute_psnr(clear_img, dehaze_1)
    psnr_2 = compute_psnr(clear_img, dehaze_2)
    
    ssim_1 = compute_ssim(clear_img, dehaze_1 , multichannel=True)
    ssim_2 = compute_ssim(clear_img, dehaze_2 , multichannel=True)
    
    mse_1 = compute_mse(clear_img.flatten(), dehaze_1.flatten())
    mse_2 = compute_mse(clear_img.flatten(), dehaze_2.flatten())
    
    fig = plt.figure(figsize=(30, 30), dpi=80)
    ax1 = fig.add_subplot(1,4,1)
    ax1.imshow(hazy_img)
    plt.axis('off')
    ax1.set_title('Hazy Image')
    ax2 = fig.add_subplot(1,4,2)
    ax2.imshow(dehaze_1)
    plt.axis('off')
    ax2.set_title(f'DCP - PSNR: {psnr_1:.3f} - SSIM: {ssim_1:.3f} - MSE: {mse_1:.3f}')

    ax3 = fig.add_subplot(1,4,3)
    ax3.imshow(dehaze_2)
    plt.axis('off')
    ax3.set_title(f'CAP - PSNR: {psnr_2:.3f} - SSIM: {ssim_2:.3f} - MSE: {mse_2:.3f}')

    ax4 = fig.add_subplot(1,4,4)
    ax4.imshow(clear_img)
    plt.axis('off')
    ax4.set_title('Clear Image')
    
    if save:  
        case_name = clear.replace('\\','/').split('/')[-1].replace('.png','')
        fig.savefig(os.path.join(result_path,case_name+'.png'))
        plt.close(fig) 
    plt.show()

In [17]:
def evaluation(hazy, clear_img, method):
    start = timeit.default_timer()
    if method == 'dcp':
        dehaze_img, t = dcp_dehaze(hazy, config['dcp'])
        print(t)
    else:
        dehaze_img = cap_dehaze(hazy, config['cap'])
    
    psnr = compute_psnr(clear_img, dehaze_img)
    ssim = compute_ssim(clear_img, dehaze_img, multichannel=True)
    mse = compute_mse(clear_img.flatten(), dehaze_img.flatten())
    end = timeit.default_timer()
    time_consume = end - start
    size = clear_img.shape[:2]
    return psnr, ssim, mse, time_consume, size

In [18]:
# cv2.imread('D:\\data\\reside\\indoor_train\\hazy\\942_9_0.74081.png').shape[:1]

In [19]:
list_hazy_imgs = glob(os.path.join(hazy_fold_path, '*.png'))
random.shuffle(list_hazy_imgs)
# list_hazy_imgs[:10]

In [21]:
list_psnr = []
list_ssim = []
list_mse = []
list_time = []
list_size = []
for i in tqdm(range(len(list_hazy_imgs[:2]))):
    hazy_img = cv2.imread(list_hazy_imgs[i])
    clear = get_clear(list_hazy_imgs[i], clear_fold_path)
    clear_img = cv2.imread(clear)
    psnr, ssim, mse, time, size = evaluation(hazy_img, clear_img, method = 'dcp')
    list_psnr.append(psnr)
    list_ssim.append(ssim)
    list_mse.append(mse)
    list_time.append(time)
    list_size.append(size)

 50%|████████████████████████████████                                | 1/2 [00:00<00:00,  1.33it/s]

0.535045282335997


100%|████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.99it/s]

0.45692731239049633


In [11]:
# clear_img

In [ ]:
np.mean(list_psnr), np.mean(list_ssim), np.mean(list_mse), np.mean(list_time), np.mean(np.array(list_size),axis=0)

In [ ]:
list_psnr = []
list_ssim = []
list_mse = []
list_time = []
list_size = []
for i in tqdm(range(len(list_hazy_imgs))):
    hazy = list_hazy_imgs[i]
    clear = get_clear(hazy, clear_fold_path)
    psnr, ssim, mse, time, size = evaluation(hazy, method = 'cap')
    list_psnr.append(psnr)
    list_ssim.append(ssim)
    list_mse.append(mse)
    list_time.append(time)
    list_size.append(size)

In [ ]:
np.mean(list_psnr), np.mean(list_ssim), np.mean(list_mse), np.mean(list_time), np.mean(np.array(list_size),axis=0)

In [ ]:
# for i in range(1,5):
#     list_hazy_imgs = glob(os.path.join(hazy_fold_path, f'{i}*.png'))
#     hazy = list_hazy_imgs[0]
#     clear = get_clear(hazy, clear_fold_path)
#     show_compaire(hazy, clear, _RESULT_PATH)
# #     plt.show()